In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src

In [ ]:
import os
import cv2
import json
import glob
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn.functional as F

from collections import Counter
from tqdm.notebook import tqdm


pd.set_option('display.width', 500)
pd.set_option('max_colwidth', 100)

In [ ]:
from params import *

from data.preparation import prepare_data

from util.plots import *
from util.logger import Config
from util.torch import load_model_weights

from model_zoo.models_seg import define_model

In [ ]:
df = prepare_data(DATA_PATH)

In [ ]:
df_train = pd.read_csv(DATA_PATH + "train.csv")

In [ ]:
df_train[df_train[df_train.columns[:6]].isna().sum(1) > 0]

In [ ]:
len(df_train), len(df.study_id.unique())

In [ ]:
import re
df['condition'] = df['condition'].apply(lambda d: d if isinstance(d, list) else [])

df['condition'] = df['condition'].apply(lambda x: [re.sub('Neural Foraminal Narrowing', 'NFN', u) for u in x])
df['condition'] = df['condition'].apply(lambda x: [re.sub('Spinal Canal Stenosis', 'SCS', u) for u in x])
df['condition'] = df['condition'].apply(lambda x: [re.sub('Subarticular Stenosis', 'SS', u) for u in x])
df['condition'] = df['condition'].apply(lambda x: [re.sub('Left', 'L', u) for u in x])
df['condition'] = df['condition'].apply(lambda x: [re.sub('Right', 'R', u) for u in x])

In [ ]:
df[df['weighting'] == "T1"].condition.apply(len).value_counts() / (len(df) / 3)

In [ ]:
df[df['weighting'] == "T2/STIR"].condition.apply(len).value_counts()

In [ ]:
df_ = pd.read_csv(DATA_PATH + "train_label_coordinates.csv")
df_ = df_[df_['condition'].apply(lambda x: 'Neural Foraminal Narrowing' in x)].reset_index(drop=True)
df_["side"] = df_['condition'].apply(lambda x: x.split()[0])

frames = pd.read_csv('../input/df_frames.csv')
frames["frames"] = frames["frames"].apply(eval)
df_ = df_.merge(frames, how="left")
df_['ordered'] = df_['frames'].apply(lambda x: x[0] < x[-1])

dfg = df_[["study_id", "series_id", "side", "instance_number", "ordered"]].groupby(["study_id", "series_id", "side"]).mean()
dfg['ordered'] = dfg['ordered'] > 0.5
dfg = dfg.reset_index().groupby(["study_id", "series_id"])

In [ ]:
df['x'] = df['coords'].apply(lambda x: [int(u[0]) for u in x] if isinstance(x, np.ndarray) else -1)

In [ ]:
df_train

In [ ]:
df

In [ ]:
df[df['study_id'] == 3637444890]

In [ ]:
df[df['study_id'] == 3637444890]

In [ ]:
for s in tqdm(df.series_id.unique()):
    dfs = df[df['series_id'] == s]

    if dfs["weighting"].values[0] != "T1":
        continue

    xs = dfs['x'].values[0]
    sides = [l[0] for l in dfs['condition'].values[0]]

    dfc = pd.DataFrame({'x': xs, 's': sides})
    dfc = dfc.groupby("s").mean()
    
    try:
        if not dfc["x"]["R"] < dfc["x"]["L"]:
            print(s)
    except KeyError:
        pass
    # display(dfc)
    # break

In [ ]:
df[df['series_id'] == 1518511736]

In [ ]:
df_train[df_train['study_id'] == 4003253].T

In [ ]:
df_train

Done ! 